In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Other imports

# Get Data


In [3]:
utilData = pd.read_csv('./Util_Run1.csv')
utilData['memLoad'] = utilData['memUse']*100/utilData['memTot']
utilData['memScore'] = utilData['memLoad']*utilData['memfreq']/825000000
utilData['memF'] = utilData['memfreq']*100/825000000
utilData['cpuScore'] = utilData['cpuload']*utilData['cpufreq']/1400000
utilData['cpuF'] = utilData['cpufreq']*100/1000000

In [4]:
# Some data modification
utilData

,memUse,memTot,memfreq,cpuid,cpuload,cpufreq,memLoad,memScore,memF,cpuScore,cpuF
0,604872,4612865,165000000,0,31,600000,13.112718,2.622544,20.000000,13.285714,60.0
1,675,28712,275000000,0,4,200000,2.350933,0.783644,33.333333,0.571429,20.0
2,1191888,12624777,413000000,0,8,200000,9.440864,4.726154,50.060606,1.142857,20.0
3,773492,8196989,275000000,0,9,300000,9.436294,3.145431,33.333333,1.928571,30.0
4,1144,57890,206000000,0,7,300000,1.976162,0.493442,24.969697,1.500000,30.0
...,...,...,...,...,...,...,...,...,...,...,...
2669,807320,11546069,413000000,0,4,200000,6.992163,3.500319,50.060606,0.571429,20.0
2670,195717,2817255,275000000,0,7,300000,6.947081,2.315694,33.333333,1.500000,30.0
2671,1344,50569,275000000,0,4,300000,2.657755,0.885918,33.333333,0.857143,30.0
2672,1344,50569,275000000,0,5,300000,2.657755,0.885918,33.333333,1.071429,30.0


In [5]:
# utilData.shift(1)
utilData.shape[0]

2674

# Fuzzy AR-MA


In [6]:
# def ART(p,endog,exog=None,data_weight=None):
    
#     df = pd.DataFrame()
#     df['endog'] = endog
#     if exog!= None:
#         df['exog'] = exog
    
    
df = pd.DataFrame()
df['Weight'] = np.ones(utilData.shape[0])
df['Value'] = utilData['cpuScore'].diff(1)
# df['Weight'] = np.ones(1000)
df['exog'] = utilData['memLoad'].shift(1)
# df.iloc[:,1]
# df['Weight']

In [30]:
from statsmodels.tools.eval_measures import rmspe
def FuzzyARX(p,df):
    df_temp = pd.DataFrame()
    df_temp['Value'] = df['Value']

    #Generating the lagged p terms
    for i in range(1,p+1):
        df_temp['Shifted_values_%d' % i ] = df_temp['Value'].shift(i)    
    df_temp['exog'] = df['exog']
    df_temp['Weight'] = df['Weight']
    train_size = (int)(1.0 * df_temp.shape[0])
    #Breaking data set into test and training
    df_train = pd.DataFrame(df_temp[0:train_size])
#     df_test = pd.DataFrame(df_temp[train_size:df.shape[0]])
    df_train_2 = df_train.dropna()
    X_train = df_train_2.iloc[:,1:p+2].values.reshape(-1,p+1)
    #Y contains the value,it is the first column
    y_train = df_train_2.iloc[:,0].values.reshape(-1,1)
    print(y_train.shape[0])
    sample_weight = df_train_2['Weight'].values
    #Running linear regression to generate the coefficents of lagged terms
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression().fit(X_train,y_train,sample_weight)
#     lr.fit(X_train,y_train,sample_weight)
    theta  = lr.coef_.T
    intercept = lr.intercept_
#     df_train_2['Predicted_Values'] = X_train.dot(lr.coef_.T) + lr.intercept_
#     df_train_2['Predicted_Values'] = lr.predict(X_train)
    # df_train_2[['Value','Predicted_Values']].plot()
    print('R-means',lr.score(X_train,y_train),'theta',lr.coef_.T,'const',lr.intercept_)
#     X_test = df_test.iloc[:,1:p+2].values.reshape(-1,p+1)
#     df_test['Predicted_Values'] = X_test.dot(lr.coef_.T) + lr.intercept_
#     df_test['Predicted_Values'] = lr.predict(X_test)
    # df_test[['Value','Predicted_Values']].plot()
#     RMSE = np.sqrt(mean_squared_error(df_test['Value'], df_test['Predicted_Values']))
#     RMSE = rmspe(df_test['Value'],df_test['Predicted_Values'])

#     print("The RMSE is :", RMSE,", Value of p : ",p)
#     return [df_train_2,df_test,theta,intercept,RMSE]


    
    print(X_train)
    import statsmodels.api as sm
    X_sm = sm.add_constant(X_train)
    print(X_sm)
#     model = sm.WLS(y_train, X_sm, weights = sample_weight)
#     results = model.fit()
#     print(results.summary())
    return [X_sm,y_train,sample_weight]

In [31]:
res = FuzzyARX(3,df)

2670
R-means 0.06906677933242333 theta [[-0.25608159]
 [-0.03690105]
 [-0.03546568]
 [ 0.045737  ]] const [-0.75122601]
[[  0.78571429   0.57142857 -12.71428571   9.43629423]
 [ -0.42857143   0.78571429   0.57142857   1.97616169]
 [ -1.07142857  -0.42857143   0.78571429   6.18651917]
 ...
 [  0.92857143  -0.71428571   0.42857143   6.94708147]
 [ -0.64285714   0.92857143  -0.71428571   2.65775475]
 [  0.21428571  -0.64285714   0.92857143   2.65775475]]
[[  1.           0.78571429   0.57142857 -12.71428571   9.43629423]
 [  1.          -0.42857143   0.78571429   0.57142857   1.97616169]
 [  1.          -1.07142857  -0.42857143   0.78571429   6.18651917]
 ...
 [  1.           0.92857143  -0.71428571   0.42857143   6.94708147]
 [  1.          -0.64285714   0.92857143  -0.71428571   2.65775475]
 [  1.           0.21428571  -0.64285714   0.92857143   2.65775475]]


In [32]:
X,y,w = res[0],res[1],res[2]
X[0]

array([  1.        ,   0.78571429,   0.57142857, -12.71428571,
         9.43629423])

In [20]:
import statsmodels.api as sm
model = sm.WLS(y, X,weights = w)
results = model.fit()
X_t = [1.0, 0.6,  -1.0, 0.9, 7.1]
# X_t = np.array(X_t).reshape(-1,4)
X_t

[1.0, 0.6, -1.0, 0.9, 7.1]

In [11]:
X_t = sm.add_constant(X_t)
X_t

array([[ 1. ,  0.6, -1. ,  0.9,  7.1],
       [ 1. ,  0.8, -0. ,  1.9,  7.4]])

In [23]:
results.get_prediction(X[0]).predicted_mean

array([-0.09101062])

In [24]:
results.get_prediction(X_t).summary_frame(0.1)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,-0.57516,0.236685,-0.964608,-0.185713,-19.064038,17.913717


In [14]:
results.pvalues

array([5.37340075e-03, 6.33829807e-39, 6.34255831e-02, 6.58770318e-02,
       2.62904968e-06])

In [15]:
results.params

array([-0.75122601, -0.25608159, -0.03690105, -0.03546568,  0.045737  ])

In [33]:
w[0]

1.0

In [16]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.069
Model:                            WLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     49.43
Date:                Tue, 18 Jul 2023   Prob (F-statistic):           3.57e-40
Time:                        14:49:30   Log-Likelihood:                -10245.
No. Observations:                2670   AIC:                         2.050e+04
Df Residuals:                    2665   BIC:                         2.053e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.7512      0.270     -2.786      0.005      -1.280      -0.223
x1            -0.2561      0.019    -13.263      0.000      -0.294      -0.218
x2            -0.0369      0.020     -1.857      0.063      -0.076       0.002
x3            -0.0355      0.019     -1.840      0.066      -0.073       0.002
x4             0.0457      0.010      4.708      0.000       0.027       0.065
==============================================================================
Omnibus:                      903.824   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            80741.570
Skew:                          -0.646   Prob(JB):                         0.00
Kurtosis:                      29.909   Cond. No.                         34.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [17]:
val = FuzzyARX(4,df)

2669
R-means 0.06906691127357123 theta [[-0.25609964]
 [-0.03692193]
 [-0.03561794]
 [-0.00055564]
 [ 0.04572275]] const [-0.75086811]
[[ -0.42857143   0.78571429   0.57142857 -12.71428571   1.97616169]
 [ -1.07142857  -0.42857143   0.78571429   0.57142857   6.18651917]
 [  0.64285714  -1.07142857  -0.42857143   0.78571429  10.52235157]
 ...
 [  0.92857143  -0.71428571   0.42857143  -0.64285714   6.94708147]
 [ -0.64285714   0.92857143  -0.71428571   0.42857143   2.65775475]
 [  0.21428571  -0.64285714   0.92857143  -0.71428571   2.65775475]]
[[  1.          -0.42857143   0.78571429   0.57142857 -12.71428571
    1.97616169]
 [  1.          -1.07142857  -0.42857143   0.78571429   0.57142857
    6.18651917]
 [  1.           0.64285714  -1.07142857  -0.42857143   0.78571429
   10.52235157]
 ...
 [  1.           0.92857143  -0.71428571   0.42857143  -0.64285714
    6.94708147]
 [  1.          -0.64285714   0.92857143  -0.71428571   0.42857143
    2.65775475]
 [  1.           0.21428571  -0

In [18]:
val = FuzzyARX(4,df)

2669
R-means 0.06906691127357123 theta [[-0.25609964]
 [-0.03692193]
 [-0.03561794]
 [-0.00055564]
 [ 0.04572275]] const [-0.75086811]
[[ -0.42857143   0.78571429   0.57142857 -12.71428571   1.97616169]
 [ -1.07142857  -0.42857143   0.78571429   0.57142857   6.18651917]
 [  0.64285714  -1.07142857  -0.42857143   0.78571429  10.52235157]
 ...
 [  0.92857143  -0.71428571   0.42857143  -0.64285714   6.94708147]
 [ -0.64285714   0.92857143  -0.71428571   0.42857143   2.65775475]
 [  0.21428571  -0.64285714   0.92857143  -0.71428571   2.65775475]]
[[  1.          -0.42857143   0.78571429   0.57142857 -12.71428571
    1.97616169]
 [  1.          -1.07142857  -0.42857143   0.78571429   0.57142857
    6.18651917]
 [  1.           0.64285714  -1.07142857  -0.42857143   0.78571429
   10.52235157]
 ...
 [  1.           0.92857143  -0.71428571   0.42857143  -0.64285714
    6.94708147]
 [  1.          -0.64285714   0.92857143  -0.71428571   0.42857143
    2.65775475]
 [  1.           0.21428571  -0

In [19]:
val = FuzzyARX(4,df)

2669
R-means 0.06906691127357123 theta [[-0.25609964]
 [-0.03692193]
 [-0.03561794]
 [-0.00055564]
 [ 0.04572275]] const [-0.75086811]
[[ -0.42857143   0.78571429   0.57142857 -12.71428571   1.97616169]
 [ -1.07142857  -0.42857143   0.78571429   0.57142857   6.18651917]
 [  0.64285714  -1.07142857  -0.42857143   0.78571429  10.52235157]
 ...
 [  0.92857143  -0.71428571   0.42857143  -0.64285714   6.94708147]
 [ -0.64285714   0.92857143  -0.71428571   0.42857143   2.65775475]
 [  0.21428571  -0.64285714   0.92857143  -0.71428571   2.65775475]]
[[  1.          -0.42857143   0.78571429   0.57142857 -12.71428571
    1.97616169]
 [  1.          -1.07142857  -0.42857143   0.78571429   0.57142857
    6.18651917]
 [  1.           0.64285714  -1.07142857  -0.42857143   0.78571429
   10.52235157]
 ...
 [  1.           0.92857143  -0.71428571   0.42857143  -0.64285714
    6.94708147]
 [  1.          -0.64285714   0.92857143  -0.71428571   0.42857143
    2.65775475]
 [  1.           0.21428571  -0